In [50]:
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import itertools as it
import pickle
import collections as ct
from collections import *
from operator import itemgetter

### leggo il file con grafo ridotto 

In [2]:
data=pd.read_csv(r'C:\Users\Daniele\Desktop\hw5\wiki-topcats-reduced.txt',sep='\t',header=None)
data.columns=['Source','Destination']
data.head()

,Source,Destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


### verifico che il grafo sia diretto sempre tramite pandas,nel secondo caso ritorna un dataframe vuoto che conferma che verso 52 non c'è alcun nodo e quindi il grafo è diretto

In [3]:
#example that show taht the graph is directd
print(data[data['Source'] ==52])
print(data[data['Destination'] ==52])

   Source  Destination
0      52       401135
1      52      1069112
2      52      1163551
Empty DataFrame
Columns: [Source, Destination]
Index: []


### creo il grafo con networkx per la visualizzazzione delle informazioni

In [4]:
G=nx.from_pandas_edgelist(data,'Destination','Source',create_using=nx.DiGraph()).to_directed()
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 461193\nNumber of edges: 2645247\nAverage in degree:   5.7357\nAverage out degree:   5.7357'

### sempre tramite nx verifico la densità del grafo

In [5]:
nx.density(G)

1.2436602635647606e-05

### creo il grafo sotto forma di dizionario che è più veloce rispetto a far girare il bfs dentro al grafo creato con newtworkx

In [6]:
grapho=data.groupby('Source')['Destination'].apply(list).to_dict()

### leggo il file delle categorie mi creo un dizionario {nome_categoria:nodi}

In [7]:
with open(r'C:\Users\Daniele\Desktop\hw5\wiki-topcats-categories.txt', 'r') as document:
    next(document)
    answer = {}
    for line in document:
        line = line.split()
        answer[line[0].replace('Category:','').replace(';','')] = line[1:]

### il dizionario di stringhe lo converto in dizionario di interi

In [9]:
for el in answer:
    answer[el] = list(map(int,answer[el]))

### filtro le categorie in modo da avere solo quelle con almeno 3500 nodi come richiesto dagli assistenti

In [11]:
filtro_answer=answer
f={ k:v for k, v in filtro_answer.items() if len(v)>=3500 }
len(f)

35

### BFS: ritorna un dizionario annidato con lo shortest path {nodo_start{nodo_end:shortest_path}}

In [33]:
def bfs_prova1(graph, root):
    seen, queue = {root}, ct.deque([(root, 0)])
    levels = {}
    
    while queue:
        vertex, level = queue.popleft()
   
        levels[vertex] = level
        for node in graph.get(vertex, []):
            if node not in seen:
                seen.add(node)
                queue.append((node, level + 1))
    return levels    

### le categorie erano composte da nodi senza cammini, quindi ho filtrato i nodi in modo da avere soltanto nodi con cammini e far andare più veloce il bfs; in questa maniera diminuiscono i nodi di tutte le categorie. Lo avevo eseguito prima di sapere come trattare tutti gli inf value.

In [14]:
#lista con solo nodi funzionanti
filter1=list(G.out_edges())
func_node= []
for i in filter1:
    func_node.append(i[0])

In [15]:
#dizionario con nodi funzionanti in ogni categoria
s = {}
for key in f.keys():
    s[key] = list(set(f[key]).intersection(set(func_node)))

### per vedere quale fosse la categoria minore ho fatto un dizionario che poi ho messo in un dataframe e mi sono fatto colorare il valore minore(da rimuovere)

In [16]:
length_dict1 = {key: len(value) for key, value in s.items()}

In [17]:
def highlight_min(data, color='yellow'):
    '''
    highlight the minimum in the DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)

In [21]:
lenght_dataframe=pd.Series(length_dict1).to_frame()
lenght_dataframe.columns=['Articles']
lenght_dataframe.style.apply(highlight_min, color='darkorange', axis=None)

,Articles
English_footballers,5656
The_Football_League_players,5907
Association_football_forwards,3515
Association_football_goalkeepers,2271
Association_football_midfielders,3771
Association_football_defenders,2989
Living_people,262035
Year_of_birth_unknown,1612
Harvard_University_alumni,4111
Major_League_Baseball_pitchers,3426


### Inizializzo un dizionario con lista vuota per ogni categoria e giro il bfs. Non salva più nulla in memoria, divide in ogni categoria gli shortest path. Qui l'ho girato per una categoria minore per lavorare sul portatile. 

In [40]:
block_ran={k:[] for k in s.keys()}
for i in tqdm(answer['People_from_Worcester']):
    tree= bfs_prova1(grapho,i)
    for k,v in s.items():
        for node in v:
            if node in tree.keys():  
                block_ran[k].append(tree[node])

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [01:04<00:00,  5.40it/s]


In [37]:
#funzione per la mediana
def median(lst):
    lst.sort()
    half = len(lst)//2  # integer division
    b = lst[half]
    c = lst[-half-1]  # for odd lengths, b == c
    return (b + c) / 2

### calcolo la mediana dal block ranking e metto tutto  in un dataframe. bisogna eliminare dal dizionario la categoria di partenza e metterla a valore 0(qui ho messo appunto 'People_from_Worcester':0.0). Va rimodificato solo il rank con il rank che avevi pensato (1/log(len(inf)) e la parte del block ranking è finita.

In [41]:
for k,v in block_ran.items():
    block_ran[k]=median(v)
block_ran.update({'People_from_Worcester':0.0})     
sorted_by_value = sorted(block_ran.items(), key=lambda kv: kv[1])
df=pd.DataFrame(sorted_by_value)
df.columns=['Category','Rank']
df

,Category,Rank
0,People_from_Worcester,0.0
1,Article_Feedback_Pilot,5.0
2,English_footballers,6.0
3,The_Football_League_players,6.0
4,Harvard_University_alumni,6.0
5,Members_of_the_United_Kingdom_Parliament_for_E...,6.0
6,English-language_albums,6.0
7,English_television_actors,6.0
8,British_films,6.0
9,English-language_films,6.0


# Node ranking

### Creo una lista di categorie con tutti i pesi di ogni articolo ; ho usato nx.subgraph di networkx e per il calcolo di tutti i pesi nx.degree    dentro a node_ranking ho una lista di defaultdict(in modo da evitare errori quando non trovava chiavi) costituita da nodo e peso {defaultdict(int,{nodo,peso})}

In [45]:
node_ranking=[]
for i in block_ran:
    sub=G.subgraph(s[i])
    temp=ct.defaultdict(int)
    for node in sub.nodes():
        temp[node]=sub.degree(node)
    node_ranking.append(temp)            

### Creo il ranking di ogni nodo; itero dentro la lista di dizionari, uso nx.predecessor per vedere i nodi in ingresso (se li trova sommo il peso del nodo attuale a quello precedente). Adesso dentro node_ranking ho  {defaultdict(int,{nodo, somma_pesi})}

In [47]:
for i in range(1,len(node_ranking)):
    for node in node_ranking[i]:
        for predecessor in G.predecessors(node):
            node_ranking[i][node] += node_ranking[i-1][predecessor]

### creo un nuovo dizionario annidato dove ho {nome_categoria{ nodo:somma pesi}}

In [51]:
node_ranking_final={}
#itero dentro alla lista di dizionari
for i in node_ranking:
    #riprendo il nome delle categorie dal precedente dizionario s
    for category in s:
        #converto il defaultDict in un normale dizionario
        node_ranking_final[category]=dict(i)
        

### creo il dizionario finale dove ho una lista di tuple per ogni categoria ordinato sui valori {categoria:[(nodo,somma pesi)]}

In [54]:
node_ranking_final_sorted={}
for k,v in node_ranking_final.items():
    order=sorted(v.items(), key=itemgetter(1),reverse=True)
    node_ranking_final_sorted[k]=order
    

In [55]:
node_ranking_final_sorted

{'American_Jews': [(1400451, 6806),
  (1400362, 4593),
  (1740428, 4589),
  (1732220, 3998),
  (1734537, 3227),
  (1740783, 2968),
  (1736740, 2119),
  (1735354, 1696),
  (1740309, 1693),
  (1741137, 1401),
  (1730797, 1198),
  (1731426, 1143),
  (1322408, 1129),
  (1740678, 1064),
  (1740925, 1024),
  (1731105, 1017),
  (1739325, 979),
  (1770798, 941),
  (1209251, 915),
  (1740502, 891),
  (1737513, 889),
  (1156828, 889),
  (1737465, 411),
  (684708, 174),
  (420262, 165),
  (420212, 162),
  (1735567, 146),
  (1737529, 130),
  (1738167, 68),
  (1735504, 62),
  (1738168, 57),
  (1740911, 51),
  (1740964, 45),
  (1738106, 41),
  (1740429, 41),
  (1740913, 40),
  (1053058, 39),
  (1053101, 39),
  (1737919, 34),
  (1738044, 34),
  (1738169, 33),
  (1738861, 32),
  (1739318, 31),
  (1741065, 31),
  (1735514, 31),
  (1740965, 30),
  (1738045, 29),
  (1740920, 29),
  (1740967, 29),
  (1733357, 29),
  (1740347, 28),
  (1738031, 27),
  (1735484, 27),
  (837422, 26),
  (1740437, 24),
  (17408